Возвращаемся к дэйтингу, некий Анатолий зарегистрировался в приложении знакомств, некий алгоритм на основе машинного
обучения порекомендовал Анатолию 30 фотографий из 100. Анатолий лайкнул 15 фотографий из 30 предложенных, 
в дальнейшем выяснилось, что Анатолий мог бы лайкнуть еще 30 фотографий из 70 непредложенных.

Будем считать, что  лайк Анатолия является положительным исходом, а отсутствие лайка - отрицательным.
Соответственно предсказанный лайк, совпавший с реальным - True positive, предсказанный лайк,
не завершившийся реальным лайком - False positive и т. д. 
Давайте оценим работу алгоритма с точки зрения метрик качества модели.

| Predicted likes | Predicted | dislikes |
| --------------- |:---------:|:--------:|
| Real likes      | 15        | 30       |
| Real dislikes   | 15        |	40       |

Такой формат представления предсказания модели и реальных исходов называется confusion matrix. Глядя на такую таблицу, легко можно понять, где именно ошибается наша модель. А также рассчитать различные метрики качества модели.

In [4]:
TP = 15
FN = 30
FP = 15
TN = 40

Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
F1 = 2 * (Precision * Recall) / (Precision + Recall)

print(f"Precision: {Precision}")
print(f"Recall: {Recall}")
print(f"F1: {F1}")


Precision: 0.5
Recall: 0.3333333333333333
F1: 0.4


Допустим, мы установили сканер отпечатков пальцев на вход в Институт биоинформатики, теперь любой студент может приложить палец и попасть внутрь. Однако сканер иногда допускает ошибки как первого так и второго рода. 

Служба охраны переживает и очень не хочет, чтобы посторонние люди были распознаны, как студенты. Какую метрику необходимо максимизировать в таком случае.

Будем считать, что

True positive - студент института был действительно распознан как студент.
False positive - не студент института был распознан как студент. 

|                          | Предсказанный студент      | Предсказанный посторонний  |
| ------------------------ |:--------------------------:|:--------------------------:|
| Реальный студент         | TP                         | FN                         |
| Реальный посторонний     | FP                         | TN                         |

Нас интересует минимизация реальных посторонних FP, FP участвует в формуле для Precision:

- [ ] False negative rate
- [ ] F1 score
- [ ] Recall
- [x] Precision

Помните, precision более важен в ситуациях, где не нужны ложные положительные срабатывания, а recall - там, где не нужны ложные отрицательные.

Например, если руководство страны пытается предотвратить эпидемию, и решило бесплатно выдавать лекарства всем заболевшим, то более важно будет покрыть всю заболевшую аудиторию == минимизировать случаи, когда больной считается здоровым == увеличить recall

Возможно чуть более понятная интерпретация этих метрик:
В поисковых системах, precision - соответствие выданной страницы запросу пользователя, а recall - доля выданных соответствующих страниц к доле всех соответствующих страниц.

In [11]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score

df = pd.read_csv("songs.csv")
df.head()

,song,year,artist,genre,lyrics,length,commas,exclamations,colons,hyphens
0,the-ownerz,2007,gangstarr,Hip-Hop,"""One-two..""\n""One-two, shots to the chin.. kno...",1300,24,5,0,11
1,dead-sound-of-misery,2006,blind-guardian,Metal,It's dark\nLet's do it now\nBring in the siren...,1566,0,0,0,0
2,modred-s-song,2006,blind-guardian,Metal,I've lost my battle before it starts\nmy first...,996,1,0,1,0
3,the-ninth-wave,2015,blind-guardian,Metal,As dark night embraces\nWe are the nation\nI w...,2868,5,0,0,0
4,skills,2014,gangstarr,Hip-Hop,"Skills, skills, skills\n""My Microphone""\n""It's...",3351,22,0,0,3


In [34]:
X = df.drop(['artist','song','year','genre','lyrics'], axis=1)
y = df.artist

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

clf = tree.DecisionTreeClassifier(max_depth=16)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')

precision

0.6638655462184874